# CSCI E-81 HW5 - Partners: Piyawan Chirayus and Cindy Liu

In [77]:
import re
import requests  # pip install requests
from bs4 import BeautifulSoup

In [72]:
BASE_URL = 'http://shakespeare.mit.edu/'

r = requests.get(BASE_URL)
assert(r.status_code == 200)

soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find_all('table')[1]
titles = {}
for link in table.find_all('a'):
    if 'Poetry' not in link.get('href'):
        if '\n' in link.string:
            link.string = link.string.replace('\n', ' ').strip()
        href = link.get('href').replace('index', 'full')
        send_url = BASE_URL + href
        print('Getting {} from {}'.format(link.string, send_url))
        r = requests.get(send_url)
        assert(r.status_code == 200)
        titles[link.string] = {
            'href': href,
            'soup': BeautifulSoup(r.content, 'html.parser')
        }


Getting All's Well That Ends Well from http://shakespeare.mit.edu/allswell/full.html
Getting As You Like It from http://shakespeare.mit.edu/asyoulikeit/full.html
Getting The Comedy of Errors from http://shakespeare.mit.edu/comedy_errors/full.html
Getting Cymbeline from http://shakespeare.mit.edu/cymbeline/full.html
Getting Love's Labours Lost from http://shakespeare.mit.edu/lll/full.html
Getting Measure for Measure from http://shakespeare.mit.edu/measure/full.html
Getting The Merry Wives of Windsor from http://shakespeare.mit.edu/merry_wives/full.html
Getting The Merchant of Venice from http://shakespeare.mit.edu/merchant/full.html
Getting A Midsummer Night's Dream from http://shakespeare.mit.edu/midsummer/full.html
Getting Much Ado About Nothing from http://shakespeare.mit.edu/much_ado/full.html
Getting Pericles, Prince of Tyre from http://shakespeare.mit.edu/pericles/full.html
Getting Taming of the Shrew from http://shakespeare.mit.edu/taming_shrew/full.html
Getting The Tempest from 

In [113]:
for title in titles:
    print('-', title, '-')
    for speaker in titles[title]['soup'].find_all(attrs={'name': re.compile('speech\d+')})[:5]:
        print(speaker.string)


- Henry VI, part 2 -
SUFFOLK
KING HENRY VI
QUEEN MARGARET
KING HENRY VI
ALL
- Pericles, Prince of Tyre -
ANTIOCHUS
PERICLES
ANTIOCHUS
PERICLES
ANTIOCHUS
- All's Well That Ends Well -
COUNTESS
BERTRAM
LAFEU
COUNTESS
LAFEU
- Hamlet -
BERNARDO
FRANCISCO
BERNARDO
FRANCISCO
BERNARDO
- The Merchant of Venice -
ANTONIO
SALARINO
SALANIO
SALARINO
ANTONIO
- Othello -
RODERIGO
IAGO
RODERIGO
IAGO
RODERIGO
- The Merry Wives of Windsor -
SHALLOW
SLENDER
SHALLOW
SLENDER
SHALLOW
- Henry VIII -
BUCKINGHAM
NORFOLK
BUCKINGHAM
NORFOLK
BUCKINGHAM
- Coriolanus -
First Citizen
All
First Citizen
All
First Citizen
- Henry V -
Chorus
CANTERBURY
ELY
CANTERBURY
ELY
- Timon of Athens -
Poet
Painter
Poet
Painter
Poet
- The Comedy of Errors -
AEGEON
DUKE SOLINUS
AEGEON
DUKE SOLINUS
AEGEON
- Titus Andronicus -
SATURNINUS
BASSIANUS
MARCUS ANDRONICUS
SATURNINUS
BASSIANUS
- Romeo and Juliet -
SAMPSON
GREGORY
SAMPSON
GREGORY
SAMPSON
- The Tempest -
Master
Boatswain
Master
Boatswain
ALONSO
- Troilus and Cressida -
TROILUS